In [1]:
from ipynb.fs.full.read_in_data import read_in_data
import json
import torch
from torchvision.transforms import transforms as T

/home/aleksandra/Documents/hackathon/knum_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("../public_dataset/reference_images_part1.json") as f:
    ref_json = json.load(f)

dir_str = "../public_dataset/reference_images_part1"
ref_data = read_in_data(dir_str, ref_json)

ref_data.head()

,id,image_id,bbox,category_id,img
12,12,4,"[665, 448, 206, 174]",8,<PIL.Image.Image image mode=RGB size=206x174 a...
13,13,4,"[475, 186, 236, 186]",8,<PIL.Image.Image image mode=RGB size=236x186 a...
14,14,4,"[962, 76, 229, 217]",8,<PIL.Image.Image image mode=RGB size=229x217 a...
15,15,4,"[778, 250, 163, 135]",8,<PIL.Image.Image image mode=RGB size=163x135 a...
16,16,4,"[184, 377, 244, 194]",8,<PIL.Image.Image image mode=RGB size=244x194 a...


In [3]:
with open("../public_dataset/images_part1_valid.json") as f:
    ref_json = json.load(f)

dir_str = "../public_dataset/images_part1_valid"
valid_data = read_in_data(dir_str, ref_json)

valid_data.head()

,id,image_id,bbox,category_id,img
4723,4723,260,"[266.71, 330.25, 204.03, 105.25]",4,<PIL.Image.Image image mode=RGB size=204x106 a...
4724,4724,260,"[307.94, 204, 280.54, 166.5]",13,<PIL.Image.Image image mode=RGB size=280x166 a...
4725,4725,261,"[457.16, 208, 132.22, 164.5]",13,<PIL.Image.Image image mode=RGB size=132x164 a...
4726,4726,261,"[421.58, 174, 126.81, 183.5]",4,<PIL.Image.Image image mode=RGB size=126x184 a...
4727,4727,261,"[266.4, 368.92, 204.19, 66.83]",4,<PIL.Image.Image image mode=RGB size=205x67 at...


In [4]:
def predict(model, ref_data, valid_data, feature_extractor=None):
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    
    to_tensor = T.ToTensor()

    ref_data = ref_data.copy()
    valid_data = valid_data.copy()
    for id_, row in ref_data.iterrows():
        if feature_extractor is not None:
             vect = model(**feature_extractor(to_tensor(row["img"]), return_tensors='pt'))
        else:
            vect = model(to_tensor(row["img"]))
            
        ref_data.at[id_, "vect"] = vect
    
    for id_, row in valid_data.iterrows():
        if feature_extractor is not None:
             vect = model(**feature_extractor(to_tensor(row["img"]), return_tensors='pt'))
        else:
            vect = model(to_tensor(row["img"]))
            
        best_row = ref_data.id.loc[0]
        min_dist = cos(ref_data.vect.loc[0], vect)
        
        for id_ref, row_ref in ref_data.iterrows():
            dist = cos(ref_data.vect.loc[id_ref], vect)
            if dist < min_dist:
                min_dist = dist
                best_row = id_ref
                
            
        valid_data[id_, "pred_category"] = ref_data.category_id[best_row]
        
    return valid_data
            
            